In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Import data

#Dataset #1
games_details = pd.read_csv ("./data/games_details.csv")
games = pd.read_csv ("./data/games.csv")
players = pd.read_csv ("./data/players.csv")
ranking = pd.read_csv ("./data/ranking.csv")
teams = pd.read_csv ("./data/teams.csv")

#Dataset #2
players_data = pd.read_csv ("./data/all_seasons.csv")

#Dataset #3
draft_data = pd.read_csv ("./data/NBA_Draft_1980_2017.tsv", sep = '\t')

C:\Users\kobyl\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
players_data = players_data[['draft_round', 'player_height', 'player_weight', 'pts', 'ast', 'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct']]
players_data = players_data.drop(players_data[players_data.draft_round == 'Undrafted'].index)#Remove undrafted
players_data['draft_round'] = pd.to_numeric(players_data['draft_round'])

In [4]:
#Using dataset #1
home = games.iloc[:,7:13]
away = games.iloc[:,14:20]
home['home'] = 1
away['home'] = 0
home = home.rename(columns={'PTS_home': 'PTS', 'FG_PCT_home': 'FG_PCT', 'FT_PCT_home': 'FT_PCT', 'FG3_PCT_home': 'FG3_PCT', 'AST_home': 'AST', 'REB_home': 'REB'})
away = away.rename(columns={'PTS_away': 'PTS', 'FG_PCT_away': 'FG_PCT', 'FT_PCT_away': 'FT_PCT', 'FG3_PCT_away': 'FG3_PCT', 'AST_away': 'AST', 'REB_away': 'REB'})
result = home.append(away, ignore_index=True)

#Normalize
scaler = MinMaxScaler()
result['PTS'] = scaler.fit_transform(result[['PTS']])
result['AST'] = scaler.fit_transform(result[['AST']])
result['REB'] = scaler.fit_transform(result[['REB']])

In [5]:
players_df = pd.read_csv ("./data/all_seasons.csv")
players_df = players_df.drop(players_df[players_df.draft_round == 'Undrafted'].index)#Remove undrafted
players_df['draft_round'] = pd.to_numeric(players_df['draft_round'])

In [6]:
# Linear Regression Predicting Net Rating
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Split the dataset into training and testing sets
X = players_df[["age", "player_height", "player_weight", "pts", "reb", "ast","oreb_pct","dreb_pct","usg_pct","ts_pct","ast_pct"]]
y = players_df["net_rating"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Fit the linear regression model to the training data
reg_model = LinearRegression().fit(X_train, y_train)

# Use the model to make predictions on the testing data
y_pred = reg_model.predict(X_test)

# Calculate the model's performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 88.61
R-squared: 0.12


In [7]:
# Linear Regression model coefficients
reg_model.coef_

array([  0.21417762,  -0.05113055,  -0.0480863 ,   0.55992148,
        -0.24968542,  -0.28075589,  31.98462897,  14.69650626,
       -43.78753428,  36.35439902,  13.94567622])

In [8]:
# QR method Linear Regression model coefficients
import numpy as np
beta_est = np.linalg.inv(X.T @ X) @ X.T @ y
beta_est

0      0.210483
1     -0.113926
2     -0.021666
3      0.547688
4     -0.175475
5     -0.152934
6     39.736897
7      8.248348
8    -41.649097
9     32.552083
10     9.607842
dtype: float64

In [9]:
# Use QR method to optimize the Linear Regression model predicting Net Rating
Q,R = np.linalg.qr(X)
beta_est = np.linalg.inv(R) @ Q.T @ y
beta_est

array([ 2.10483494e-01, -1.13926027e-01, -2.16662769e-02,  5.47688092e-01,
       -1.75474761e-01, -1.52933739e-01,  3.97368974e+01,  8.24834818e+00,
       -4.16490975e+01,  3.25520835e+01,  9.60784156e+00])

In [10]:
# Compute new y from QR method Linear Regression
y_pred_qr = np.dot(X, beta_est)

# Calculate the QR method performance metrics
mse = mean_squared_error(y, y_pred_qr)
r2 = r2_score(y, y_pred_qr)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 93.07
R-squared: 0.19


In [11]:
# Random Forest Regression Predicting Net Rating
from sklearn.ensemble import RandomForestRegressor

# Fit a random forest regression model to the training data
rf_model = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)

# Use the model to make predictions on the testing data
y_pred = rf_model.predict(X_test)

# Calculate the model's performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 87.21
R-squared: 0.13


In [12]:
# Support Vector Regression Predicting Net Rating
from sklearn.svm import SVR

# Fit a support vector regression model to the training data
svr_model = SVR(kernel='rbf', C=100, gamma=0.1).fit(X_train, y_train)

# Use the model to make predictions on the testing data
y_pred = svr_model.predict(X_test)

# Calculate the model's performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 126.34
R-squared: -0.26


In [13]:
# Gradient Boosting Regression Predicting Net Rating
from sklearn.ensemble import GradientBoostingRegressor

# Fit a gradient boosting regression model to the training data
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42).fit(X_train, y_train)

# Use the model to make predictions on the testing data
y_pred = gb_model.predict(X_test)

# Calculate the model's performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 86.07
R-squared: 0.14
